# Install sklearn

In [38]:
!pip install -U scikit-learn
!pip3 install -U scikit-learn

zsh:1: command not found: pip


## Naive Bayes Model

In [101]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

data = pd.read_csv('trg.csv')
vectorizer = CountVectorizer()
matrix = vectorizer.fit_transform(data.abstract)

data = data.rename(columns={'class': 'class_label'})
X = data.drop('class_label', axis=1)
y = data['class_label']
data  = pd.DataFrame(data= matrix.toarray(), columns = vectorizer.get_feature_names_out())
print(X)
# data.head()

        id                                           abstract
0        1  the 4 202 353 bp genome of the alkaliphilic ba...
1        2  the complete 1751377-bp sequence of the genome...
2        3  in 1992 we started assembling an ordered libra...
3        4  the aim of this study is to measure human mito...
4        5  the amino acid sequence of the spirulina maxim...
...    ...                                                ...
3995  3996  we have isolated and characterized two diureti...
3996  3997  myotonias are muscle diseases in which the fun...
3997  3998  cysteine synthase o-acetylserine sulfhydrylase...
3998  3999  a region of 25 nucleotides is highly conserved...
3999  4000  thermoanaerobacter tengcongensis is a rod-shap...

[4000 rows x 2 columns]


In [108]:
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X['abstract'], y, test_size=0.4, random_state=42)

class NaiveBayes:
    def __init__(self, alpha=1):
        self.alpha = alpha
        self.word_counts = defaultdict(lambda: defaultdict(int))
        self.class_counts = defaultdict(int)
        self.vocab = set()

    def fit(self, X, y):
        for abstract, cls in zip(X, y):
            self.class_counts[cls] += 1
            for word in abstract.split():
                self.word_counts[cls][word] += 1
                self.vocab.add(word)

        self.vocab_size = len(self.vocab)
        total_count = len(y)
        self.class_priors = {cls: np.log(count) - np.log(total_count) for cls, count in self.class_counts.items()}

    def predict(self, X):
        predictions = []
        for abstract in X:
            class_posteriors = {cls: self.class_priors[cls] for cls in self.class_counts}
            words = abstract.split()
            for cls in self.class_counts:
                for word in words:
                    if word in self.vocab:
                        word_count = self.word_counts[cls].get(word, 0) + self.alpha
                        smoothed_count = np.log(word_count) - np.log(self.class_counts[cls] + self.vocab_size * self.alpha)
                        class_posteriors[cls] += smoothed_count
            predictions.append(max(class_posteriors, key=class_posteriors.get))
        return predictions

nb_clf = NaiveBayes()
nb_clf.fit(X_train, y_train)
y_pred = nb_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.81
